In [1]:
# Текст, который нужно сохранить
text = """Tourists found the octagonal lighthouse after a two-mile hike through fog
In twenty forty-nine, neural implants became standard for city workers
He whispered something in Icelandic I could not quite translate
Lucia balanced a porcelain vase on her elbow without flinching
They gathered quietly beneath the turbine as the wind picked up speed"""

# Сохранение текста в файл test_sentences.txt
with open("test_sentences.txt", "w", encoding="utf-8") as file:
    file.write(text)

print("Файл успешно сохранён как 'test_sentences.txt'")

Файл успешно сохранён как 'test_sentences.txt'


In [2]:
!pip install -q TTS

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 57.9 MB/s eta 0:00:00:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 99.3 MB/s eta 0:00:00:00:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 100.2 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 56.9 MB/s eta 0:00:00:00:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 111.8 MB/s eta 0:00:0000:010:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━

In [3]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchaudio
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from tqdm import tqdm
import librosa
import random
from pathlib import Path
import pickle

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

class LJSpeechDataset(Dataset):
    def __init__(self, data_dir, metadata_file, segment_length=8192, hop_length=256, 
                 n_mels=80, sample_rate=22050, max_items=None):
        self.data_dir = Path(data_dir)
        self.segment_length = segment_length
        self.hop_length = hop_length
        self.n_mels = n_mels
        self.sample_rate = sample_rate
        
        # Load metadata
        self.metadata = pd.read_csv(metadata_file, sep='|', header=None, 
                                  names=['filename', 'transcription', 'normalized_transcription'])
        
        if max_items:
            self.metadata = self.metadata.head(max_items)
        
        # Filter out files that don't exist
        self.valid_files = []
        for _, row in self.metadata.iterrows():
            wav_path = self.data_dir / 'wavs' / f"{row['filename']}.wav"
            if wav_path.exists():
                self.valid_files.append(row['filename'])
        
        print(f"Found {len(self.valid_files)} valid audio files")
        
        # Mel spectrogram transform
        self.mel_transform = torchaudio.transforms.MelSpectrogram(
            sample_rate=sample_rate,
            n_fft=1024,
            hop_length=hop_length,
            n_mels=n_mels,
            f_min=0,
            f_max=sample_rate // 2
        )
    
    def __len__(self):
        return len(self.valid_files)
    
    def __getitem__(self, idx):
        filename = self.valid_files[idx]
        wav_path = self.data_dir / 'wavs' / f"{filename}.wav"
        
        # Load audio
        waveform, sr = torchaudio.load(wav_path)
        if sr != self.sample_rate:
            resampler = torchaudio.transforms.Resample(sr, self.sample_rate)
            waveform = resampler(waveform)
        
        # Convert to mono if stereo
        if waveform.shape[0] > 1:
            waveform = torch.mean(waveform, dim=0, keepdim=True)
        
        # Ensure exact segment length
        if waveform.shape[1] > self.segment_length:
            start_idx = random.randint(0, waveform.shape[1] - self.segment_length)
            waveform = waveform[:, start_idx:start_idx + self.segment_length]
        else:
            # Pad if too short
            pad_length = self.segment_length - waveform.shape[1]
            waveform = F.pad(waveform, (0, pad_length))
        
        # Ensure waveform is exactly segment_length
        waveform = waveform[:, :self.segment_length]
        
        # Generate mel spectrogram
        mel_spec = self.mel_transform(waveform)
        mel_spec = torch.log(mel_spec + 1e-5)  # Log mel spectrogram
        
        # Calculate expected mel frames based on hop_length
        expected_frames = self.segment_length // self.hop_length
        
        # Trim or pad mel spectrogram to expected frames
        if mel_spec.shape[-1] > expected_frames:
            mel_spec = mel_spec[..., :expected_frames]
        elif mel_spec.shape[-1] < expected_frames:
            pad_frames = expected_frames - mel_spec.shape[-1]
            mel_spec = F.pad(mel_spec, (0, pad_frames))
        
        return {
            'audio': waveform.squeeze(0),  # [segment_length]
            'mel': mel_spec.squeeze(0)     # [n_mels, expected_frames]
        }

class ResidualBlock(nn.Module):
    def __init__(self, channels, kernel_size=3, dilation=1):
        super().__init__()
        self.conv1 = nn.Conv1d(channels, channels, kernel_size, 
                              padding=dilation*(kernel_size-1)//2, dilation=dilation)
        self.conv2 = nn.Conv1d(channels, channels, kernel_size, 
                              padding=dilation*(kernel_size-1)//2, dilation=dilation)
        self.relu = nn.LeakyReLU(0.2)
        
    def forward(self, x):
        residual = x
        x = self.relu(self.conv1(x))
        x = self.conv2(x)
        return x + residual

class Generator(nn.Module):
    def __init__(self, n_mels=80, ngf=32, n_residual_blocks=3):
        super().__init__()
        
        # Initial convolution
        self.conv_pre = nn.Conv1d(n_mels, ngf * 8, 7, padding=3)
        
        # Upsampling layers
        self.upsampling = nn.ModuleList([
            nn.ConvTranspose1d(ngf * 8, ngf * 4, 16, stride=8, padding=4),  # 8x upsample
            nn.ConvTranspose1d(ngf * 4, ngf * 2, 16, stride=8, padding=4),  # 8x upsample  
            nn.ConvTranspose1d(ngf * 2, ngf, 4, stride=2, padding=1),       # 2x upsample
            nn.ConvTranspose1d(ngf, ngf, 4, stride=2, padding=1),           # 2x upsample
        ])
        
        # Residual blocks for each upsampling layer
        self.res_blocks = nn.ModuleList([
            nn.ModuleList([ResidualBlock(ngf * 4, dilation=3**i) for i in range(n_residual_blocks)]),
            nn.ModuleList([ResidualBlock(ngf * 2, dilation=3**i) for i in range(n_residual_blocks)]),
            nn.ModuleList([ResidualBlock(ngf, dilation=3**i) for i in range(n_residual_blocks)]),
            nn.ModuleList([ResidualBlock(ngf, dilation=3**i) for i in range(n_residual_blocks)]),
        ])
        
        # Multi-band output layers (4 subbands)
        self.conv_post = nn.ModuleList([
            nn.Conv1d(ngf, 1, 7, padding=3) for _ in range(4)
        ])
        
        self.relu = nn.LeakyReLU(0.2)
        self.tanh = nn.Tanh()
        
    def forward(self, mel):
        x = self.conv_pre(mel)
        
        for i, (upsample, res_blocks) in enumerate(zip(self.upsampling, self.res_blocks)):
            x = self.relu(upsample(x))
            
            # Apply residual blocks
            for res_block in res_blocks:
                x = res_block(x)
        
        # Generate 4 subbands
        subbands = []
        for conv_out in self.conv_post:
            subband = self.tanh(conv_out(x))
            subbands.append(subband)
        
        return torch.cat(subbands, dim=1)  # [batch, 4, time]

class SubbandDiscriminator(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.convs = nn.ModuleList([
            nn.Conv1d(1, 16, 15, padding=7),
            nn.Conv1d(16, 64, 41, stride=4, padding=20, groups=4),
            nn.Conv1d(64, 256, 41, stride=4, padding=20, groups=16),
            nn.Conv1d(256, 1024, 41, stride=4, padding=20, groups=64),
            nn.Conv1d(1024, 1024, 41, stride=4, padding=20, groups=256),
            nn.Conv1d(1024, 1024, 5, padding=2),
            nn.Conv1d(1024, 1, 3, padding=1),
        ])
        
        self.relu = nn.LeakyReLU(0.2)
    
    def forward(self, x):
        feature_maps = []
        
        for i, conv in enumerate(self.convs):
            x = conv(x)
            if i < len(self.convs) - 1:
                x = self.relu(x)
                feature_maps.append(x)
        
        return x, feature_maps

class MultibandDiscriminator(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Discriminators for each subband + full-band
        self.discriminators = nn.ModuleList([
            SubbandDiscriminator() for _ in range(5)  # 4 subbands + 1 full-band
        ])
    
    def forward(self, y):
        # y shape: [batch, 4, time] for subbands or [batch, 1, time] for full-band
        outputs = []
        feature_maps = []
        
        if y.shape[1] == 4:  # Multi-band input
            # Process each subband
            for i in range(4):
                subband = y[:, i:i+1, :]  # [batch, 1, time]
                out, fmaps = self.discriminators[i](subband)
                outputs.append(out)
                feature_maps.append(fmaps)
            
            # Full-band (sum of subbands)
            full_band = torch.sum(y, dim=1, keepdim=True)
            out, fmaps = self.discriminators[4](full_band)
            outputs.append(out)
            feature_maps.append(fmaps)
        else:  # Single-band input
            out, fmaps = self.discriminators[4](y)
            outputs.append(out)
            feature_maps.append(fmaps)
        
        return outputs, feature_maps

def multiband_stft_loss(y_true, y_pred, n_ffts=[1024, 2048, 512], hop_lengths=None):
    """Multi-resolution STFT loss"""
    if hop_lengths is None:
        hop_lengths = [n_fft // 4 for n_fft in n_ffts]
    
    total_loss = 0.0
    
    # Ensure both tensors have the same length
    min_length = min(y_true.shape[-1], y_pred.shape[-1])
    y_true_aligned = y_true[..., :min_length]
    y_pred_aligned = y_pred[..., :min_length]
    
    for n_fft, hop_length in zip(n_ffts, hop_lengths):
        # STFT
        stft_true = torch.stft(y_true_aligned.squeeze(1), n_fft=n_fft, hop_length=hop_length, 
                              return_complex=True, normalized=True)
        stft_pred = torch.stft(y_pred_aligned.squeeze(1), n_fft=n_fft, hop_length=hop_length, 
                              return_complex=True, normalized=True)
        
        # Magnitude loss
        mag_true = torch.abs(stft_true)
        mag_pred = torch.abs(stft_pred)
        mag_loss = F.l1_loss(mag_pred, mag_true)
        
        # Spectral convergence loss  
        sc_loss = torch.norm(mag_true - mag_pred, p='fro') / (torch.norm(mag_true, p='fro') + 1e-6)
        
        total_loss += mag_loss + sc_loss
    
    return total_loss

def feature_matching_loss(real_fmaps, fake_fmaps):
    """Feature matching loss between discriminator feature maps"""
    loss = 0.0
    
    for real_fmap_list, fake_fmap_list in zip(real_fmaps, fake_fmaps):
        for real_fmap, fake_fmap in zip(real_fmap_list, fake_fmap_list):
            # Align temporal dimensions by taking minimum length
            min_length = min(real_fmap.size(-1), fake_fmap.size(-1))
            real_fmap_aligned = real_fmap[..., :min_length]
            fake_fmap_aligned = fake_fmap[..., :min_length]
            loss += F.l1_loss(fake_fmap_aligned, real_fmap_aligned.detach())
    
    return loss

def discriminator_loss(real_outputs, fake_outputs):
    """Discriminator loss (hinge loss)"""
    loss = 0.0
    
    for real_out, fake_out in zip(real_outputs, fake_outputs):
        real_loss = torch.mean(F.relu(1.0 - real_out))
        fake_loss = torch.mean(F.relu(1.0 + fake_out))
        loss += real_loss + fake_loss
    
    return loss

def generator_loss(fake_outputs):
    """Generator adversarial loss"""
    loss = 0.0
    
    for fake_out in fake_outputs:
        loss += -torch.mean(fake_out)
    
    return loss

def train_multiband_melgan(data_dir, output_dir, num_epochs=200, batch_size=32, lr_g=2e-4, lr_d=1e-4):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Training on device: {device}")
    
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    
    # Dataset and DataLoader
    dataset = LJSpeechDataset(
        data_dir=data_dir,
        metadata_file=os.path.join(data_dir, 'metadata.csv'),
        max_items=2000  # Limit for Kaggle - remove for full training
    )
    
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, 
                           num_workers=2, pin_memory=True)
    
    # Models
    generator = Generator().to(device)
    discriminator = MultibandDiscriminator().to(device)
    
    # Optimizers
    opt_g = optim.Adam(generator.parameters(), lr=lr_g, betas=(0.5, 0.9))
    opt_d = optim.Adam(discriminator.parameters(), lr=lr_d, betas=(0.5, 0.9))
    
    # Learning rate schedulers
    scheduler_g = optim.lr_scheduler.ExponentialLR(opt_g, gamma=0.999)
    scheduler_d = optim.lr_scheduler.ExponentialLR(opt_d, gamma=0.999)
    
    # Training history
    history = {'g_loss': [], 'd_loss': [], 'stft_loss': []}
    
    print("Starting training...")
    
    for epoch in range(num_epochs):
        epoch_g_loss = 0.0
        epoch_d_loss = 0.0
        epoch_stft_loss = 0.0
        
        progress_bar = tqdm(dataloader, desc=f'Epoch {epoch+1}/{num_epochs}')
        
        for batch_idx, batch in enumerate(progress_bar):
            audio = batch['audio'].to(device)  # [batch, time]
            mel = batch['mel'].to(device)      # [batch, n_mels, frames]
            
            batch_size = audio.shape[0]
            
            # ================== Train Discriminator ==================
            opt_d.zero_grad()
            
            # Generate fake audio (multi-band)
            with torch.no_grad():
                fake_audio_mb = generator(mel)  # [batch, 4, time]
            
            # Real audio (single band)
            real_audio = audio.unsqueeze(1)  # [batch, 1, time]
            
            # Discriminator on real
            real_outputs, real_fmaps = discriminator(real_audio)
            
            # Discriminator on fake multi-band
            fake_outputs, fake_fmaps = discriminator(fake_audio_mb.detach())
            
            # Discriminator loss
            d_loss = discriminator_loss(real_outputs, fake_outputs)
            d_loss.backward()
            opt_d.step()
            
            # ================== Train Generator ==================
            opt_g.zero_grad()
            
            # Generate fake audio
            fake_audio_mb = generator(mel)
            
            # Discriminator on fake (for generator training)
            fake_outputs, fake_fmaps_g = discriminator(fake_audio_mb)
            
            # Generator adversarial loss
            g_adv_loss = generator_loss(fake_outputs)
            
            # Feature matching loss
            fm_loss = feature_matching_loss(real_fmaps, fake_fmaps_g)
            
            # STFT loss (convert multi-band to single-band for comparison)
            fake_audio_sb = torch.sum(fake_audio_mb, dim=1, keepdim=True)
            stft_loss = multiband_stft_loss(real_audio, fake_audio_sb)
            
            # Total generator loss
            g_loss = g_adv_loss + 45.0 * fm_loss + 45.0 * stft_loss
            g_loss.backward()
            opt_g.step()
            
            # Update metrics
            epoch_g_loss += g_loss.item()
            epoch_d_loss += d_loss.item()
            epoch_stft_loss += stft_loss.item()
            
            # Update progress bar
            progress_bar.set_postfix({
                'G_Loss': f'{g_loss.item():.4f}',
                'D_Loss': f'{d_loss.item():.4f}',
                'STFT_Loss': f'{stft_loss.item():.4f}'
            })
        
        # Update learning rates
        scheduler_g.step()
        scheduler_d.step()
        
        # Average losses for the epoch
        avg_g_loss = epoch_g_loss / len(dataloader)
        avg_d_loss = epoch_d_loss / len(dataloader)
        avg_stft_loss = epoch_stft_loss / len(dataloader)
        
        history['g_loss'].append(avg_g_loss)
        history['d_loss'].append(avg_d_loss)
        history['stft_loss'].append(avg_stft_loss)
        
        print(f"Epoch {epoch+1}: G_Loss: {avg_g_loss:.4f}, D_Loss: {avg_d_loss:.4f}, STFT_Loss: {avg_stft_loss:.4f}")
        
        # Save checkpoints
        if (epoch + 1) % 10 == 0:
            checkpoint = {
                'epoch': epoch + 1,
                'generator_state_dict': generator.state_dict(),
                'discriminator_state_dict': discriminator.state_dict(),
                'optimizer_g_state_dict': opt_g.state_dict(),
                'optimizer_d_state_dict': opt_d.state_dict(),
                'history': history
            }
            
            torch.save(checkpoint, os.path.join(output_dir, f'checkpoint_epoch_{epoch+1}.pth'))
            print(f"Checkpoint saved at epoch {epoch+1}")
        
        # Generate sample audio every 20 epochs
        if (epoch + 1) % 20 == 0:
            generate_sample_audio(generator, dataset, device, output_dir, epoch+1)
    
    # Save final model
    torch.save(generator.state_dict(), os.path.join(output_dir, 'multiband_melgan_generator.pth'))
    torch.save(discriminator.state_dict(), os.path.join(output_dir, 'multiband_melgan_discriminator.pth'))
    
    # Plot training history
    plot_training_history(history, output_dir)
    
    print("Training completed!")
    return generator, discriminator, history

def generate_sample_audio(generator, dataset, device, output_dir, epoch):
    """Generate sample audio for evaluation"""
    generator.eval()
    
    with torch.no_grad():
        # Get a sample from dataset
        sample = dataset[0]
        mel = sample['mel'].unsqueeze(0).to(device)
        original_audio = sample['audio'].numpy()
        
        # Generate audio
        fake_audio_mb = generator(mel)
        fake_audio = torch.sum(fake_audio_mb, dim=1)  # Convert to single band
        fake_audio = fake_audio.squeeze().cpu().numpy()
        
        # Save generated audio
        sample_dir = os.path.join(output_dir, 'samples')
        os.makedirs(sample_dir, exist_ok=True)
        
        # Save as wav file (you may need to install soundfile: pip install soundfile)
        try:
            import soundfile as sf
            sf.write(os.path.join(sample_dir, f'generated_epoch_{epoch}.wav'), 
                    fake_audio, 22050)
            sf.write(os.path.join(sample_dir, f'original_epoch_{epoch}.wav'), 
                    original_audio, 22050)
        except ImportError:
            print("soundfile not available, saving as numpy arrays")
            np.save(os.path.join(sample_dir, f'generated_epoch_{epoch}.npy'), fake_audio)
            np.save(os.path.join(sample_dir, f'original_epoch_{epoch}.npy'), original_audio)
    
    generator.train()

def plot_training_history(history, output_dir):
    """Plot and save training history"""
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 3, 1)
    plt.plot(history['g_loss'], label='Generator Loss')
    plt.title('Generator Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    
    plt.subplot(1, 3, 2)
    plt.plot(history['d_loss'], label='Discriminator Loss')
    plt.title('Discriminator Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    
    plt.subplot(1, 3, 3)
    plt.plot(history['stft_loss'], label='STFT Loss')
    plt.title('STFT Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'training_history.png'))
    plt.close()

In [47]:
# ================== MAIN TRAINING SCRIPT ==================
if __name__ == "__main__":
    # Configuration
    DATA_DIR = "/kaggle/input/the-lj-speech-dataset/LJSpeech-1.1/"
    OUTPUT_DIR = "/kaggle/working/multiband_melgan_output"
    
    # Hyperparameters
    NUM_EPOCHS = 400  
    BATCH_SIZE = 64  
    LR_G = 5e-4
    LR_D = 1e-5
    
    print("Multi-band MelGAN Training Configuration:")
    print(f"Data Directory: {DATA_DIR}")
    print(f"Output Directory: {OUTPUT_DIR}")
    print(f"Number of Epochs: {NUM_EPOCHS}")
    print(f"Batch Size: {BATCH_SIZE}")
    print(f"Generator Learning Rate: {LR_G}")
    print(f"Discriminator Learning Rate: {LR_D}")
    print("-" * 50)
    
    # Start training
    generator, discriminator, history = train_multiband_melgan(
        data_dir=DATA_DIR,
        output_dir=OUTPUT_DIR,
        num_epochs=NUM_EPOCHS,
        batch_size=BATCH_SIZE,
        lr_g=LR_G,
        lr_d=LR_D
    )
    
    print("Training completed successfully!")
    print(f"Models saved in: {OUTPUT_DIR}")
    
    # Optionally, test with your TextToSpecConverter
    print("\nTesting integration with TextToSpecConverter...")
    
    # Load your text-to-mel converter
    
    try:
        t2s = TextToSpecConverter()
        
        mel_spec = t2s.text2spec(text)  # shape (80, T)
        print("Mel spec shape:", mel_spec.shape)
        # Transpose mel_spec to shape (80, 1719)
        mel_spec = mel_spec.T
        
        # Then convert to tensor and add batch dimension
        mel_tensor = torch.FloatTensor(mel_spec).unsqueeze(0).to(next(generator.parameters()).device)
        
        print("Mel tensor shape before generator:", mel_tensor.shape)  # Should be [1, 80, 1719]
        
        with torch.no_grad():
            fake_audio_mb = generator(mel_tensor)
        
            # Save result
            import soundfile as sf
            sf.write(os.path.join(OUTPUT_DIR, 'text_to_speech_result.wav'), fake_audio, 22050)
            print(f"Text-to-speech result saved: {OUTPUT_DIR}/text_to_speech_result.wav")
            
    except Exception as e:
        print(f"Text-to-speech integration test failed: {e}")
        print("You can manually test the trained generator with your TextToSpecConverter later.")

Multi-band MelGAN Training Configuration:
Data Directory: /kaggle/input/the-lj-speech-dataset/LJSpeech-1.1/
Output Directory: /kaggle/working/multiband_melgan_output
Number of Epochs: 400
Batch Size: 64
Generator Learning Rate: 0.0005
Discriminator Learning Rate: 1e-05
--------------------------------------------------
Training on device: cuda
Found 1000 valid audio files
Starting training...


Epoch 1/400: 100%|██████████| 16/16 [00:15<00:00,  1.04it/s, G_Loss=142.7229, D_Loss=1.6752, STFT_Loss=2.8963]


Epoch 1: G_Loss: 253.6703, D_Loss: 1.8498, STFT_Loss: 5.3644


Epoch 2/400: 100%|██████████| 16/16 [00:15<00:00,  1.06it/s, G_Loss=136.8671, D_Loss=1.2720, STFT_Loss=2.7525]


Epoch 2: G_Loss: 149.9432, D_Loss: 1.4641, STFT_Loss: 3.0499


...

Epoch 398: G_Loss: 76.2309, D_Loss: 0.0000, STFT_Loss: 1.3002


Epoch 399/400: 100%|██████████| 16/16 [00:15<00:00,  1.06it/s, G_Loss=76.1546, D_Loss=0.0000, STFT_Loss=1.2989]


Epoch 399: G_Loss: 77.0490, D_Loss: 0.0000, STFT_Loss: 1.3184


Epoch 400/400: 100%|██████████| 16/16 [00:15<00:00,  1.06it/s, G_Loss=79.4549, D_Loss=0.0000, STFT_Loss=1.3716]


Epoch 400: G_Loss: 76.7685, D_Loss: 0.0000, STFT_Loss: 1.3121
Checkpoint saved at epoch 400
Training completed!
Training completed successfully!
Models saved in: /kaggle/working/multiband_melgan_output

Testing integration with TextToSpecConverter...
 > tts_models/en/ljspeech/fast_pitch is already downloaded.
 > vocoder_models/en/ljspeech/hifigan_v2 is already downloaded.
 > Using model: fast_pitch
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True


In [35]:
#!/usr/bin/env python3
import torch
import torchaudio
from TTS.api import TTS
from TTS.tts.utils.synthesis import synthesis


class TextToSpecConverter:
    def __init__(self, model_name: str = "tts_models/en/ljspeech/fast_pitch"):
        self.model_name = model_name
        self.tts_handler = TTS(model_name=model_name)
        print('**' * 25,
              'INFO: discard default vocoder initialization messages above',
              '**' * 25, sep='\n')
        self.model = self.tts_handler.synthesizer.tts_model
        self.config = self.tts_handler.synthesizer.tts_config
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.use_cuda = True if self.device == "cuda" else False
        print(f"Model {model_name} loaded")
    
    def text2spec(self, text: str):
        """
        Convert text to mel spectrogram using pretrained TTS model.
        Args:
            text (str): Input text to convert to mel spectrogram
        Returns:
            mel_spec (numpy.ndarray): Mel spectrogram of the input text
                with shape [C, T] = [num_mel_channels, num_frames]
        """
        outputs = synthesis(
            self.model,
            text,
            self.config,
            use_cuda=False,
            use_griffin_lim=False,
            do_trim_silence=False
        )
        mel_spec = outputs["outputs"]["model_outputs"][0].detach().cpu().numpy()
        # denormalize tts output based on the tts audio config
        mel_spec = self.model.ap.denormalize(mel_spec.T).T
        return mel_spec


def save_spectrogram(mel_spec, filename="spectrogram.png", title="Mel Spectrogram"):
    """
    Save the mel spectrogram as an image file
    
    Args:
        mel_spec: The mel spectrogram to save
        filename: Output filename
        title: Title for the plot
    """
    import matplotlib.pyplot as plt
    
    plt.figure(figsize=(10, 4))
    plt.imshow(mel_spec, aspect='auto', origin='lower')
    plt.colorbar(format='%+2.0f dB')
    plt.title(title)
    plt.xlabel('Frames')
    plt.ylabel('Mel Channels')
    plt.tight_layout()
    plt.savefig(filename, bbox_inches='tight')
    plt.close()
    print(f"Spectrogram saved to {filename}")


def melspec_to_audio_default_vocoder(t2s, mel_spec, filename='output.wav'):
    vocoder_input = t2s.tts_handler.synthesizer.vocoder_ap.normalize(mel_spec.T)
    vocoder_input = torch.tensor(vocoder_input).unsqueeze(0)
    # [1, C, T]
    waveform = t2s.tts_handler.synthesizer.vocoder_model.inference(vocoder_input.to("cpu"))
    waveform = waveform.squeeze(0)
    torchaudio.save(filename, waveform.cpu(), 22050)
    print(f"Audio saved to {filename}")


if __name__ == "__main__":
    t2s = TextToSpecConverter()
    mel_spec = t2s.text2spec(text)
    
    save_spectrogram(mel_spec, filename="output_spectrogram.png", title="Mel Spectrogram of 'Hello, world!'")

    # If you want to validate mel_spec with default vocoder, use the code sample below
    melspec_to_audio_default_vocoder(t2s, mel_spec, filename='output.wav')

 > tts_models/en/ljspeech/fast_pitch is already downloaded.
 > vocoder_models/en/ljspeech/hifigan_v2 is already downloaded.
 > Using model: fast_pitch
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:2.718281828459045
 | > hop_length:256
 | > win_length:1024
 > Vocoder Model: hifigan
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resam